In [1]:
import networkx as nx

from bokeh.io import output_file, show, output_notebook
from bokeh.plotting import figure, from_networkx, ColumnDataSource

from bokeh.models import (Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,CustomJS)
from bokeh.palettes import Spectral4
from bokeh.models import Legend, LegendItem

import pandas as pd


In [2]:
df = pd.read_csv("data.csv")

reshape = []

for i, j, in df.iterrows():
    for _, k in df.iterrows():
        if (j['code'] == k['code']):
            pass
        elif j['group'] == 'nan':
            reshape.append({'code1':j['code'],
                       'code2': j['code'],
                       'group': 'None'})
        elif (j['group'] == k['group']):
            reshape.append({'code1': j['code'],
                       'code2': k['code'],
                       'group': j['group']})
        else:
            pass
df1 = pd.DataFrame(reshape)
df1

,code1,code2,group
0,110A,110B,fruit
1,110B,110A,fruit
2,220A,220B,meat
3,220A,220C,meat
4,220B,220A,meat
5,220B,220C,meat
6,220C,220A,meat
7,220C,220B,meat
8,300A,310A,vegetable
9,300A,320A,vegetable


In [3]:
g = nx.from_pandas_edgelist(df1, source='code1', target='code2')
groups = dict(zip(df1.code1, df1.group))


nx.set_node_attributes(g, groups, name="group")
colormap = {"fruit": "yellow", "meat": "red", "vegetable": "green"}
colors = {k: colormap[v] for k, v in groups.items()}
nx.set_node_attributes(g, colors, name="color")


In [4]:
code = '''if (cb_data.index.indices.length > 0) { 
                const index = cb_data.index.indices[0];
                
                if (source.data.group[index] === "meat") {
                    hover.tooltips = [["Group", "@group"]];  
                }
                else {
                    hover.tooltips = [["Name", "@group"]];
                }                                     
            } else {
                hover.tooltips = [["Hello", "World"]];
            }'''

hover = HoverTool()
hover.callback = CustomJS(args = dict(source = graph_renderer.node_renderer.data_source, hover = hover), code = code)
#hover.renderers=[graph.node_renderer]

#hover = HoverTool(
#    tooltips=None,
#    renderers=[graph.node_renderer],
#    callback=CustomJS(code=code)
#)

NameError: name 'graph_renderer' is not defined

In [5]:
graph_renderer.node_renderer.data_source

NameError: name 'graph_renderer' is not defined

In [38]:
plot = Plot()
plot.title.text = 'Bokeh Plot'

#hover = HoverTool(tooltips=[("Group", "@group")])

graph_renderer = from_networkx(g, nx.spring_layout, scale = 1, center = (0,0))

# manipulating nodes
graph_renderer.node_renderer.glyph = Circle(size = 15, fill_color = "color")
graph_renderer.node_renderer.selection_glyph = Circle(size = 15, fill_color = 
"color")
graph_renderer.node_renderer.hover_glyph = Circle(size = 15, fill_color = 
 Spectral4[1])
graph_renderer.node_renderer.data_source.data['group'] = [groups[g] for g in graph_renderer.node_renderer.data_source.data["index"]]

# manipulating edges
graph_renderer.edge_renderer.glyph = MultiLine(line_color = '#CCCCCC', line_alpha = 
 .5, line_width = 5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color = Spectral4[2], 
 line_width = 5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color = Spectral4[1], 
 line_width = 5)



graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph_renderer)

code = '''
    if (cb_data.index.indices.length > 0) {
        const index = cb_data.index.indices[0];

        if (source.data.group[index] === "meat") {
            hover.tooltips = [["Group", "@group"]];  
        }
        else {
            hover.tooltips = [["Name", "@group"]];
        }                                     
    }
'''

hover = HoverTool(
    renderers=[graph_renderer.node_renderer]
)
hover.callback = CustomJS(
    args = dict(source = graph_renderer.node_renderer.data_source, hover = hover),
    code = code
)

plot.add_tools(hover, TapTool())

output_notebook()
show(plot)

Loading BokehJS ...

In [130]:
#https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html#customjs-for-hover